In [1]:
import os
import os.path
os.chdir('..')

In [2]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np
from matplotlib import cm, colors
import pytorch_lightning as pl

from simclr_finetuner import SSLFineTuner
from simclr_module import SimCLR
from pointnet_module import SupervisedPointNet
from datasets.shapenet_parts.shapenet_parts import ShapeNetParts
from datasets.data_modules import PartSegmentationDataModule

from transforms import FineTuningTestDataTransform
from augmentations.augmentations import Rotation, Flip, Rescale, GaussianNoise

/home/shepherd/anaconda3/lib/python3.7/site-packages/pl_bolts/utils/warnings.py:32: UserWarning: You want to use `wandb` which is not installed yet, install it with `pip install wandb`.
  f' install it with `pip install {pypi_name}`.' + extra_text
/home/shepherd/anaconda3/lib/python3.7/site-packages/pl_bolts/utils/warnings.py:32: UserWarning: You want to use `gym` which is not installed yet, install it with `pip install gym`.
  f' install it with `pip install {pypi_name}`.' + extra_text


In [ ]:
batch_size = 4
#transforms = [Rotation(0.5),Flip(1),Rescale(),GaussianNoise(0.7)]
transforms = [Rescale(0.5)]

backbone_ckpt = 'checkpoints/simclr_best_21071416584428.ckpt'
ssl_finetuner_ckpt = 'checkpoints/fine_tuning_best_21071514122946.ckpt'

backbone = SimCLR.load_from_checkpoint(backbone_ckpt)

for transform in transforms:
    current_transform = FineTuningTestDataTransform([transform])
    dm = PartSegmentationDataModule(batch_size,test_transforms=current_transform,fine_tuning=True)
    seg_class_map = dm.seg_class_map
    test_dataloader = dm.test_dataloader()
    model = SSLFineTuner.load_from_checkpoint(ssl_finetuner_ckpt, backbone=backbone, 
                                   seg_class_map=seg_class_map)    
    
    trainer = pl.Trainer(
        gpus=1,
        num_nodes=1,
        precision=16,
        max_epochs=1,
        distributed_backend=None,
        sync_batchnorm=False,
    )
    results = trainer.test(model = model, test_dataloaders = test_dataloader)
    print('Results for transform: {} \n{}'.format(str(transform),results))

In [3]:
# For supervised
batch_size = 4
#transforms = [Rotation(0.5),Flip(1),Rescale(0.5),GaussianNoise(0.7)]
transforms = [Flip(1),Rescale(0.5),GaussianNoise(0.7)]

#supervised_no_aug_ckpt = 'checkpoints/supervised_no_aug_best_21071320395666.ckpt'
supervised_ckpt = 'checkpoints/supervised_aug_best_21071615280008.ckpt'

for transform in transforms:
    current_transform = FineTuningTestDataTransform([transform])
    dm = PartSegmentationDataModule(batch_size,test_transforms=current_transform,num_workers=8,fine_tuning=True)
    seg_class_map = dm.seg_class_map
    test_dataloader = dm.test_dataloader()    
    model = SupervisedPointNet.load_from_checkpoint(supervised_ckpt,seg_class_map=seg_class_map)
    
    trainer = pl.Trainer(
        gpus=1,
        num_nodes=1,
        precision=16,
        max_epochs=1,
        distributed_backend=None,
        sync_batchnorm=False,
    )
    results = trainer.test(model = model, test_dataloaders = test_dataloader)
    print('Results for transform: {} \n{}'.format(str(transform),results))

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 100%|██████████| 719/719 [02:27<00:00,  4.86it/s]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'eval mIoU of Airplane': 0.7025840282440186,
 'eval mIoU of Bag': 0.5056630373001099,
 'eval mIoU of Cap': 0.5080223083496094,
 'eval mIoU of Car': 0.42773231863975525,
 'eval mIoU of Chair': 0.8219462037086487,
 'eval mIoU of Earphone': 0.47785118222236633,
 'eval mIoU of Guitar': 0.8449558019638062,
 'eval mIoU of Knife': 0.5751577019691467,
 'eval mIoU of Lamp': 0.5795390009880066,
 'eval mIoU of Laptop': 0.3886492848396301,
 'eval mIoU of Motorbike': 0.2593928575515747,
 'eval mIoU of Mug': 0.6609437465667725,
 'eval mIoU of Pistol': 0.5860928297042847,
 'eval mIoU of Rocket': 0.40793415904045105,
 'eval mIoU of Skateboard': 0.4997321665287018,
 'eval mIoU of Table': 0.7234246730804443,
 'test_accuracy': 0.840644121170044,
 'test_class_avg_accuracy': 0.6011771559715271,
 'test_class_avg_iou': 0.5606013536453247,
 'tes

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 100%|██████████| 719/719 [03:23<00:00,  3.54it/s]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'eval mIoU of Airplane': 0.7649351954460144,
 'eval mIoU of Bag': 0.5866702795028687,
 'eval mIoU of Cap': 0.6495068669319153,
 'eval mIoU of Car': 0.6352315545082092,
 'eval mIoU of Chair': 0.8573871850967407,
 'eval mIoU of Earphone': 0.5885202884674072,
 'eval mIoU of Guitar': 0.8658721446990967,
 'eval mIoU of Knife': 0.8136282563209534,
 'eval mIoU of Lamp': 0.7450048327445984,
 'eval mIoU of Laptop': 0.9432350993156433,
 'eval mIoU of Motorbike': 0.4587348997592926,
 'eval mIoU of Mug': 0.8508864641189575,
 'eval mIoU of Pistol': 0.7154349684715271,
 'eval mIoU of Rocket': 0.4333653748035431,
 'eval mIoU of Skateboard': 0.6953000426292419,
 'eval mIoU of Table': 0.7912290096282959,
 'test_accuracy': 0.9101462960243225,
 'test_class_avg_accuracy': 0.7615200281143188,
 'test_class_avg_iou': 0.7121838927268982,
 'test_

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 100%|██████████| 719/719 [03:38<00:00,  3.29it/s]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'eval mIoU of Airplane': 0.7455281019210815,
 'eval mIoU of Bag': 0.5810840725898743,
 'eval mIoU of Cap': 0.6356973052024841,
 'eval mIoU of Car': 0.6212706565856934,
 'eval mIoU of Chair': 0.8529841303825378,
 'eval mIoU of Earphone': 0.5838450789451599,
 'eval mIoU of Guitar': 0.8588099479675293,
 'eval mIoU of Knife': 0.7992196679115295,
 'eval mIoU of Lamp': 0.74406898021698,
 'eval mIoU of Laptop': 0.939895510673523,
 'eval mIoU of Motorbike': 0.4424215257167816,
 'eval mIoU of Mug': 0.8483887314796448,
 'eval mIoU of Pistol': 0.708850622177124,
 'eval mIoU of Rocket': 0.4300008714199066,
 'eval mIoU of Skateboard': 0.6753106117248535,
 'eval mIoU of Table': 0.7887049913406372,
 'test_accuracy': 0.9071887135505676,
 'test_class_avg_accuracy': 0.7510631084442139,
 'test_class_avg_iou': 0.7035050392150879,
 'test_inst